## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-04-10-08-31-22.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [8]:
# Define list of desired months (excluding current month)
desired_months = ['February']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [139]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    1784
Name: PRICE, dtype: int64
-------
False    1320
True      464
Name: $/SQUARE FEET, dtype: int64
-------
False    1637
True      147
Name: YEAR BUILT, dtype: int64
-------


In [140]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [141]:
# df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
# df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
# df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
# df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
# df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [142]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1064,PAST SALE,March-1-2023,Condo/Co-op,3806 W Washington Blvd Unit DG,Chicago,IL,60624.0,25000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/3806-W-Washi...,MRED,11698586.0,N,Y,41.882121,-87.721154
416,PAST SALE,March-31-2023,Condo/Co-op,13707 S Stewart Ave Unit A1W,Riverdale,IL,60827.0,34000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Riverdale/13707-S-St...,MRED,11732753.0,N,Y,41.646143,-87.630270
1129,PAST SALE,March-8-2023,Condo/Co-op,917 E 78th St Unit 202E,Chicago,IL,60619.0,35000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/917-E-78th-S...,MRED,11698436.0,N,Y,41.752863,-87.601985
782,PAST SALE,March-16-2023,Condo/Co-op,7337 S South Shore Dr #311,Chicago,IL,60649.0,40000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/7337-S-South...,MRED,11713585.0,N,Y,41.763318,-87.558357
1394,PAST SALE,March-1-2023,Condo/Co-op,2604 E State St Unit 1B,Burnham,IL,60633.0,45000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Burnham/2604-E-State...,MRED,11632529.0,N,Y,41.630229,-87.554746
444,PAST SALE,March-12-2023,Condo/Co-op,852 E 82nd St Unit 852-1,Chicago,IL,60619.0,46000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/852-E-82nd-S...,MRED,11719849.0,N,Y,41.745927,-87.603065
977,PAST SALE,March-3-2023,Condo/Co-op,7854 S South Shore Dr #306,Chicago,IL,60649.0,49900.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/7854-S-South...,MRED,11701712.0,N,Y,41.752719,-87.549552
1193,PAST SALE,March-6-2023,Condo/Co-op,22510 Jackson Ct Unit 2D,Richton Park,IL,60471.0,52000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Richton-Park/22510-J...,MRED,11695319.0,N,Y,41.480805,-87.708760
1650,PAST SALE,March-14-2023,Condo/Co-op,22513 Jackson Ct Unit 3F,Richton Park,IL,60471.0,55000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Richton-Park/22513-J...,MRED,11639674.0,N,Y,41.480805,-87.708761
204,PAST SALE,March-28-2023,Condo/Co-op,4280 W Ford City Dr #404,Chicago,IL,60652.0,57000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/4280-W-Ford-...,MRED,11729636.0,N,Y,41.753945,-87.730531


In [143]:
print(df_filtered['URL'].iloc[416])

https://www.redfin.com/IL/Riverdale/13707-S-Stewart-Ave-60827/unit-A1W/home/12759074


In [144]:
# # Correct the prices, if needed
# df_filtered.at[618,'PRICE']=(295000)
# df_filtered.at[832,'PRICE']=(355000)

In [145]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [146]:
# # Corrections, if needed
# df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
# df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [147]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
217,6901 S Oglesby Ave Unit 10C,Chicago,27.0
264,8364 S Brandon Ave,Chicago,29.0
1064,3806 W Washington Blvd Unit DG,Chicago,36.0
204,4280 W Ford City Dr #404,Chicago,48.0
1394,2604 E State St Unit 1B,Burnham,56.0
416,13707 S Stewart Ave Unit A1W,Riverdale,57.0
1351,8216 S Jeffery Ave Unit 3W,Chicago,59.0
780,2306 Holiday Ter #145,Lansing,60.0
1719,102 N Hamlin Ave Unit A3,Chicago,62.0
1718,1942 E 74th St Unit 1C,Chicago,63.0


In [148]:
print(df_filtered.URL.iloc[217])

https://www.redfin.com/IL/Chicago/6901-S-Oglesby-Ave-60649/unit-10C/home/45515507


In [149]:
# Corrections, if needed
#
#
#

In [150]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [151]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [152]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [153]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [154]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [155]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [156]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"March 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [157]:
m.save('index.html')

## Summary Info

In [158]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [159]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [160]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [161]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Lincoln Park 2550, 2550 N Lakeview Ave Unit S801 Chicago | Price $4,400,000 | $1,110 psf | Year built: 2012
Least Expensive
CHI - West Garfield Park, 3806 W Washington Blvd Unit DG Chicago | Price $25,000 | $36 psf | Year built: nan
Highest Price Per Square Foot
CHI - Near North Side, 14 W Superior St #5304 Chicago | Price $3,775,000 | $1,258 psf | Year built: 2022
Lowest Price Per Square Foot
CHI - South Shore, 6901 S Oglesby Ave Unit 10C Chicago | Price $70,000 | $27 psf | Year built: 1929
Newest
CHI - Lincoln Park, 1823 N Sedgwick St Unit PHS Chicago | Price $2,550,000 | $nan psf | Year built: 2023
Oldest
CHI - Near West Side, 1000 W Washington Blvd #301 Chicago | Price $905,000 | $393 psf | Year built: 1882


## Time on Market Calculator

In [162]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [163]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 15) ## List (Earlier) date
date2 = datetime(2023, 3, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

105


## Map URL Snagger

In [164]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [165]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCounty_condo_sales_month_ending_march_2023


## Get Summary Data

In [166]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 1784
--------
Total sale price: $663,220,457
--------
Median sale price: $275,000
--------
Max sale price: $4,400,000
--------
Min sale price: $25,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,258
--------
Min price per square foot: $27
--------
Median price per square foot: $231
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1882.0
----------
Average building age: 1977.679291386683


## To CSV

In [167]:
df_filtered.to_csv(f'{desired_months}_condo.csv')